## Naive Bayes

### References:
* http://www.analyticsvidhya.com/blog/2015/09/naive-bayes-explained/

In [159]:
%run "../common.ipynb"

In [163]:
fileName="../data/pima-indians-diabetes.csv"

df  = LoadDataSet(fileName);
displayDFs([df], maxrows=12)


count,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000
mean,3.845,120.895,69.105,20.536,79.799,31.993,0.472,33.241,0.349
std,3.370,31.973,19.356,15.952,115.244,7.884,0.331,11.760,0.477
min,0.000,0.000,0.000,0.000,0.000,0.000,0.078,21.000,0.000
25%,1.000,99.000,62.000,0.000,0.000,27.300,0.244,24.000,0.000
50%,3.000,117.000,72.000,23.000,30.500,32.000,0.372,29.000,0.000
75%,6.000,140.250,80.000,32.000,127.250,36.600,0.626,41.000,1.000
max,17.000,199.000,122.000,99.000,846.000,67.100,2.420,81.000,1.000
,,,,,,,,,
,Num_Pregnant (int64),Glucose_c (int64),BP (int64),SkinThick (int64),Serum_Insulin (int64),BMI (float64),Diabetes (float64),Age (int64),Diabetic (int64)
0,6,148,72,35,0,33.600,0.627,50,1


In [266]:
# Example of Naive Bayes implemented from Scratch in Python
import csv
import random
import math

def loadCsv(filename):
    lines = csv.reader(open(filename, "rt"))
    dataset = list(lines)
    dataset = dataset[3:]
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]

def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

def mean(numbers):
	return sum(numbers)/float(len(numbers))

def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	summaries = [(np.array(a).mean(), np.array(a).std(), np.array(a).var()) for a in zip(*dataset)]
	del summaries[-1]
	return summaries

def summarizeByClass(dataset):
	separated = separateByClass(dataset)
	summaries = {}
	for classValue, instances in separated.items():
		summaries[classValue] = summarize(instances)
	return summaries

def calculateProbability(x, mean, stdev, var=None):
    if (var is None):
        var = math.pow(stdev,2)
    exponent = math.exp(-(math.pow(x-mean,2)/(2*var)))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev, var = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev, var)
	return probabilities
			
def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel

def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions

def getProbs(summaries, testSet):
    r = []
    for i in range(len(testSet)):
        p = calculateClassProbabilities(summaries, testSet[i])
        r.append(p)
    return r


def getAccuracy(testSet, predictions):
	correct = 0
	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

filename = '../data/pima-indians-diabetes.csv'
splitRatio = 0.67
dataset = loadCsv(filename)
trainingSet, testSet = splitDataset(dataset, splitRatio)
print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)) )
# prepare model
summaries = summarizeByClass(trainingSet)
# test model

testSet = testSet[0:50]
predictions = getPredictions(summaries, testSet)

probs = getProbs(summaries, testSet)
    
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: {0}%'.format(accuracy))

Split 768 rows into train=514 and test=254 rows
Accuracy: 80.0%


In [267]:
s = [a for a in zip(*dataset)]
np.array(s[0]).mean()
np.array(s[0]).var()

11.339272393120657

### Using Python tools

In [268]:
def SplitXy(Xy):
    Xy10=Xy[0:8]
    Xy10 = Xy;
    #print Xy10
    #print "========"
    zXy10=list(zip(*Xy10))
    y= zXy10[-1]
    del zXy10[-1]
    z1=zip(*zXy10)
    X=[list(t) for t in z1]
    return X,y

from sklearn.naive_bayes import GaussianNB
X,y = SplitXy(trainingSet) 
Xt,yt = SplitXy(testSet)

model = GaussianNB()
model.fit(X, y)


GaussianNB()

In [269]:
### Compare the models built by Python

print ("Class: 0")
for i,j in enumerate(model.theta_[0]):
    print ("({:8.2f} {:9.2f} {:7.2f} )".format(j, model.sigma_[0][i], sqrt(model.sigma_[0][i])) , end="")
    print ("==> ", summaries[0][i])
    
print ("Class: 1")
for i,j in enumerate(model.theta_[1]):
    print ("({:8.2f} {:9.2f} {:7.2f} )".format(j, model.sigma_[1][i], sqrt(model.sigma_[1][i])) , end="")
    print ("==> ", summaries[1][i])

Class: 0
(    3.24      9.28    3.05 )==>  (3.2447761194029852, 3.0468620305914031, 9.2833682334595675)
(  109.99    704.71   26.55 )==>  (109.98507462686567, 26.546322530998598, 704.7072399198039)
(   67.68    337.36   18.37 )==>  (67.677611940298505, 18.36732839871183, 337.35875250612611)
(   20.25    228.90   15.13 )==>  (20.247761194029852, 15.129336513739021, 228.89682334595679)
(   72.78  11033.82  105.04 )==>  (72.776119402985074, 105.04197041716266, 11033.815549120074)
(   30.59     64.52    8.03 )==>  (30.59014925373134, 8.0323594594107899, 64.518798485186011)
(    0.44      0.09    0.31 )==>  (0.43777014925373142, 0.30575883239335033, 0.09348846358654489)
(   31.12    130.79   11.44 )==>  (31.116417910447762, 11.436320734604895, 130.78943194475383)
Class: 1
(    4.87     13.46    3.67 )==>  (4.8715083798882679, 3.6693237566253765, 13.463936830935364)
(  141.45   1100.46   33.17 )==>  (141.44692737430168, 33.173174008514692, 1100.4594737991947)
(   69.97    507.60   22.53 )==>

In [270]:
Pt = model.predict_proba(Xt)
Pt[0:10]

array([[ 0.98,  0.02],
       [ 0.93,  0.07],
       [ 0.86,  0.14],
       [ 0.1 ,  0.9 ],
       [ 0.03,  0.97],
       [ 0.55,  0.45],
       [ 0.53,  0.47],
       [ 0.9 ,  0.1 ],
       [ 0.08,  0.92],
       [ 0.53,  0.47]])

In [271]:
p1 = model.predict(Xt)
for i in range(len(predictions)):
    if (not predictions[i] == p1[i] ):
        print ("{} differ {} {}, {} ".format(i, predictions[i] , p1[i] , Xt[i]), end="\n")
        break;

5 differ 1.0 0.0, [7.0, 100.0, 0.0, 0.0, 0.0, 30.0, 0.484, 32.0] 


In [272]:
probs[0:10]

[{0.0: 2.6647212180382806e-12, 1.0: 1.1650326590182071e-13},
 {0.0: 1.4705978178754816e-12, 1.0: 1.9166842219694805e-13},
 {0.0: 6.1589986096661238e-13, 1.0: 1.8292304549930851e-13},
 {0.0: 1.3829725011762472e-14, 1.0: 2.3947635545536097e-13},
 {0.0: 3.7749573507409827e-17, 1.0: 2.6415178743713595e-15},
 {0.0: 1.5008905259338552e-15, 1.0: 2.2904670348751916e-15},
 {0.0: 3.7522950177107856e-14, 1.0: 6.1616147230491638e-14},
 {0.0: 4.7974987035374041e-12, 1.0: 9.6078342822872527e-13},
 {0.0: 4.7954328725175488e-15, 1.0: 9.9285549859707505e-14},
 {0.0: 3.0899900451065483e-13, 1.0: 5.0945170367497614e-13}]

In [273]:
X,y = SplitXy(trainingSet)
yy=np.array(y)
y0=yy[yy==0]
y1=yy[yy==1]
p0=len(y0)/len(y)
p1=len(y1)/len(y)
print(len(y0), len(y1), len(y), p0, p1)

335 179 514 0.6517509727626459 0.34824902723735407


In [274]:
from dill.source import getsource
print(getsource(model.predict_log_proba))

    def predict_log_proba(self, X):
        """
        Return log-probability estimates for the test vector X.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]

        Returns
        -------
        C : array-like, shape = [n_samples, n_classes]
            Returns the log-probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute `classes_`.
        """
        jll = self._joint_log_likelihood(X)
        # normalize by P(x) = P(f_1, ..., f_n)
        log_prob_x = logsumexp(jll, axis=1)
        return jll - np.atleast_2d(log_prob_x).T



In [260]:
model.predict_log_proba(Xt)[0:10]

array([[ -1.14e-02,  -4.48e+00],
       [ -2.39e+00,  -9.65e-02],
       [ -2.93e-04,  -8.13e+00],
       [ -8.55e-01,  -5.54e-01],
       [ -8.10e-01,  -5.89e-01],
       [ -1.23e-01,  -2.15e+00],
       [ -3.53e-01,  -1.21e+00],
       [ -1.24e+00,  -3.43e-01],
       [ -1.69e+00,  -2.03e-01],
       [ -1.88e+00,  -1.66e-01]])

In [261]:
jll = model._joint_log_likelihood(Xt)